In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
diningtbDF = pd.read_csv('./project/dining_table.csv', index_col=0)
diningtbDF['image'] 

0       ./static/img/kitchen/table/table1.png
1       ./static/img/kitchen/table/table2.png
2       ./static/img/kitchen/table/table3.png
3       ./static/img/kitchen/table/table4.png
4       ./static/img/kitchen/table/table5.png
                       ...                   
94     ./static/img/kitchen/table/table98.png
95     ./static/img/kitchen/table/table99.png
96    ./static/img/kitchen/table/table100.png
97    ./static/img/kitchen/table/table101.png
98    ./static/img/kitchen/table/table102.png
Name: image, Length: 99, dtype: object

In [4]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(diningtbDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

diningtbDF['nouns'] = box

100%|██████████| 99/99 [00:22<00:00,  4.50it/s]


In [5]:
diningtbDF.head(1)

,brand,item,price,image,link,review,nouns
0,프리메이드,원형 티 카페 테이블 2size,"39,900",./static/img/kitchen/table/table1.png,https://ohou.se/productions/123785/selling?aff...,['몇달 사용하고 올립니다~ 싸이즈도 혼자 또는 둘이 밥먹고 술마시기 너무 좋고 튼...,달 사용 싸이 도 혼자 둘 밥 술 마시기 여기저기 동시 사용 가성 비 최고 조립 다...


In [6]:
# 중간 저장
diningtbDF.to_csv('diningtbDF.csv')

In [7]:
diningtbDF = pd.read_csv('./diningtbDF.csv', index_col=0)
diningtbDF.head(1)

,brand,item,price,image,link,review,nouns
0,프리메이드,원형 티 카페 테이블 2size,"39,900",./static/img/kitchen/table/table1.png,https://ohou.se/productions/123785/selling?aff...,['몇달 사용하고 올립니다~ 싸이즈도 혼자 또는 둘이 밥먹고 술마시기 너무 좋고 튼...,달 사용 싸이 도 혼자 둘 밥 술 마시기 여기저기 동시 사용 가성 비 최고 조립 다...


In [8]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(diningtbDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_diningtb = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_diningtb

,가가,가걱,가게,가격,가계부,가공,가구,가구라,가구점,가기,...,힌지,힐것,힐링,힘껏,힘덜,힘썻어,힘음,힘좀,힙니,힝구핑
0,0,0,0,13,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,10,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,5,0,0,25,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,13,0,0,12,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,0,20,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95,0,0,0,15,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,5,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,22,0,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# dtm 파일 저장
dtm_diningtb.to_csv('dtm_diningtb.csv')

---

In [11]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

diningtbDF = pd.read_csv('./diningtbDF.csv', index_col=0)
dtm_diningtb = pd.read_csv('./dtm_diningtb.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = '앉아도 꺼지지 않는 폭식한 소파'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(diningtbDF['brand'])
item_list = np.array(diningtbDF['item'])
price_list = np.array(diningtbDF['price'])
image_list = np.array(diningtbDF['image'])

result_cosine = make_cosine(dtm_diningtb, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

체나레 밀카 HPM 반타원형 카페 화이트 라운드 테이블 1400 2colors 176,900 ./static/img/kitchen/table/table32.png


In [12]:
# 위의 코드 함수화
def find_diningtb(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    diningtbDF = pd.read_csv('./diningtbDF.csv', index_col=0)
    dtm_diningtb = pd.read_csv('./dtm_diningtb.csv', index_col=0)

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(diningtbDF['brand'])
    item_list = np.array(diningtbDF['item'])
    price_list = np.array(diningtbDF['price'])
    image_list = np.array(diningtbDF['image'])

    result_cosine = make_cosine(dtm_diningtb, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]